# Netzwerk aus Dokumenten aufgrund ihrer Ähnlichkeit

- Wir probieren nun was ganz anderes. Nämlich ein Netzwerk aus Dokumenten zu bauen aufgrund ihrer Ähnlichkeit zueinander. 
- Die Idee dahinter ist Dokumente visuell zu clustern und so zu erkennnen welche ähnlichen Dokumente zusammengehören. 
- Die Idee ähnliche Dinge als Netzwerk zu visualisieren ist sehr praktisch und mächtig weil sie uns erlaubt in allen Dingen Muster zu sehen. 
- Wir probieren das mal anhand der Berichterstattung auf der Fronpage von swissinfo. 

# Artikel von der Front Page scrapen
- Zuerst scrapen wir die Artikel von der Frontpage

In [ ]:
import requests
from bs4 import BeautifulSoup
import nltk

In [ ]:
# Get 100 Frontpage Items
frontpage_url = "https://www.swissinfo.ch/webservice/swi-eng-2.0/overview"
r = requests.get(frontpage_url).json()
items = []
urls = []
for item in r["page"]["item"][0]["content"]:
    try:
        tmp = requests.get("https://www.swissinfo.ch/webservice/swi-eng-2.0/detail%s" % item["url"])
        text = BeautifulSoup(tmp.json()["htmldetail"], "lxml").get_text()
        items.append(text)
        urls.append(item["canonical"])
        print("Done: %s" % item["url"])
    except:
        pass

In [ ]:
items[1][0:300]

In [ ]:
urls[1]

# Dokumentenähnlichkeit bestimmen
- Work tokenization
- Stemming
- Stopword removal
- (Clustering with k-means)

In [ ]:
import collections
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint
import seaborn as sns

def word_tokenizer(text):
        #tokenizes and stems the text
        tokens = word_tokenize(text)
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(t) for t in tokens if t not in stopwords.words('english')]
        return tokens


def cluster_items(items, nb_of_clusters=5,mds=False):
        tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenizer,
                                        stop_words=stopwords.words('english'),
                                        lowercase=True)
        #builds a tf-idf matrix for the sentences
        tfidf_matrix = tfidf_vectorizer.fit_transform(items)
        
        # Show matrix
        pairwise_similarity = tfidf_matrix * tfidf_matrix.T
        ax = sns.heatmap(pairwise_similarity.A)
        ax.plot()
        
        # Clustering
        kmeans = KMeans(n_clusters=nb_of_clusters)
        kmeans.fit(tfidf_matrix)
        clusters = collections.defaultdict(list)
        for i, label in enumerate(kmeans.labels_):
                clusters[label].append(i)
        if mds:
            return [pairwise_similarity,dict(clusters)]
        else:
            return dict(clusters)


#items = ["Nature is beautiful","I like green apples", "We should protect the trees","Fruit trees provide fruits","Green apples are tasty"]

#nclusters= 3
#parwise_similarity, clusters = cluster_items(items, nclusters,True)

# Baby example: Wie funktioniert das nochmal?

Wie sieht das Resultat aus für:
- 2 Sätze mit nature
- 2 Sätze mit apple
- 1 Satz mit apple and nature
- 1 Satz mit computers

In [ ]:
nclusters = 3
baby_items = ["Nature is beautiful",
              "We should protect nature",
              "I like green apples", 
              "Green apples are tasty",
              "Apples and nature are fun.",
              "Technical sentence with computers."]
result,clusters = cluster_items(baby_items,nclusters,True)

In [ ]:
clusters

In [ ]:
for cluster in range(nclusters):
        print "cluster ",cluster,":"
        for i,item in enumerate(clusters[cluster]):
                print "\titem ",i,": ",baby_items[item]

## Jetzt die Ähnlichkeit der Artikel auf der Frontpage bestimmen

In [ ]:
result,clusters = cluster_items(items,nclusters,True)
for cluster in range(nclusters):
        print "cluster ",cluster,":"
        for i,item in enumerate(clusters[cluster]):
                print "\turl ",urls[i].split("/")[-3],": ",urls[i].split("/")[-2]

# Als Netzwerk exportieren

In [ ]:
import networkx as nx
G  = nx.Graph(name="Texts")

for url in urls:
    url_clean = url.split("/")[-2]
    G.add_node(url, size=1, label=url)

entries = result.todense().tolist()

for i,row in enumerate(urls):
    row = row.split("/")[-2]
    for j,col in enumerate(urls):
        col = col.split("/")[-2]
        if entries[i][j] > 0.2:
            G.add_edge(row,col,weight=entries[i][j])

nx.write_gexf(G,"Texts.gexf")